In [98]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn import preprocessing

In [120]:
df = pd.read_csv('data/data.csv')

In [121]:
basic_stats = pd.DataFrame()

In [122]:
cols = [
 'Winner',
 'B_age',
 'B_Height_cms',
 'B_Reach_cms',
 'B_Weight_lbs',
 'R_Height_cms',
 'R_Reach_cms',
 'R_Weight_lbs',
 'R_age']

#removing stance and weight class for now cuz lazy


#  'weight_class',
#  'R_Stance',
    
#  'B_Stance',

# 'R_fighter',
#  'B_fighter',

In [123]:
basic_stats = df[cols]

In [124]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 9 columns):
Winner          5144 non-null object
B_age           4972 non-null float64
B_Height_cms    5136 non-null float64
B_Reach_cms     4478 non-null float64
B_Weight_lbs    5138 non-null float64
R_Height_cms    5140 non-null float64
R_Reach_cms     4828 non-null float64
R_Weight_lbs    5141 non-null float64
R_age           5080 non-null float64
dtypes: float64(8), object(1)
memory usage: 361.8+ KB


# Replacing all nulls with bullshit filler vals for ez-pz first model

In [125]:
basic_stats["B_age"] = basic_stats["B_age"].fillna(basic_stats["B_age"].mean())
basic_stats["R_age"] = basic_stats["R_age"].fillna(basic_stats["R_age"].mean())

basic_stats["B_Height_cms"] = basic_stats["B_Height_cms"].fillna(basic_stats["B_Height_cms"].mean())
basic_stats["R_Height_cms"] = basic_stats["R_Height_cms"].fillna(basic_stats["R_Height_cms"].mean())

basic_stats["B_Weight_lbs"] = basic_stats["B_Weight_lbs"].fillna(basic_stats["B_Weight_lbs"].mean())
basic_stats["R_Weight_lbs"] = basic_stats["R_Weight_lbs"].fillna(basic_stats["R_Weight_lbs"].mean())

basic_stats["B_Reach_cms"] = basic_stats["B_Reach_cms"].fillna(basic_stats["B_Reach_cms"].mean())
basic_stats["R_Reach_cms"] = basic_stats["R_Reach_cms"].fillna(basic_stats["R_Reach_cms"].mean())

# basic_stats["B_Stance"] = basic_stats["B_Stance"].fillna("Orthodox")

# basic_stats["R_Stance"] = basic_stats["R_Stance"].fillna("Orthodox")


/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [126]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 9 columns):
Winner          5144 non-null object
B_age           5144 non-null float64
B_Height_cms    5144 non-null float64
B_Reach_cms     5144 non-null float64
B_Weight_lbs    5144 non-null float64
R_Height_cms    5144 non-null float64
R_Reach_cms     5144 non-null float64
R_Weight_lbs    5144 non-null float64
R_age           5144 non-null float64
dtypes: float64(8), object(1)
memory usage: 361.8+ KB


In [127]:
basic_stats.head()

,Winner,B_age,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age
0,Red,31.0,167.64,170.18,135.0,162.56,162.56,135.0,32.0
1,Red,32.0,167.64,167.64,125.0,165.10,167.64,125.0,31.0
2,Red,36.0,185.42,185.42,155.0,180.34,193.04,155.0,35.0
3,Blue,26.0,170.18,170.18,135.0,162.56,172.72,135.0,29.0
4,Blue,32.0,180.34,185.42,250.0,187.96,190.50,264.0,26.0


In [128]:
basic_stats["Winner"] = basic_stats["Winner"].apply(lambda x: 1 if x == "Red" else 0)

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [129]:
# one_hot = pd.get_dummies((basic_stats["R_Stance"])
# basic_stats = basic_stats.drop('R_stance',axis = 1)
# basic_stats = basic_stats.join(one_hot)

In [130]:
basic_stats.head()

,Winner,B_age,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age
0,1,31.0,167.64,170.18,135.0,162.56,162.56,135.0,32.0
1,1,32.0,167.64,167.64,125.0,165.10,167.64,125.0,31.0
2,1,36.0,185.42,185.42,155.0,180.34,193.04,155.0,35.0
3,0,26.0,170.18,170.18,135.0,162.56,172.72,135.0,29.0
4,0,32.0,180.34,185.42,250.0,187.96,190.50,264.0,26.0


In [131]:
y = basic_stats["Winner"]

In [133]:
basic_stats = basic_stats.drop("Winner",axis = 1)

In [135]:
X = basic_stats

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 0)

In [137]:
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [138]:
clf.score(X_test,y_test)

0.6547433903576982